### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
import datetime
from sqlalchemy import create_engine

In [ ]:
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, encoding="utf8", engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, encoding="utf8", engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [ ]:
# cell for work

movies.head()

In [ ]:
len(movies)

In [ ]:
# cell for work
reviews.head()

In [ ]:
# more work
len(reviews)

In [ ]:
genres = []

for val in movies.genre:
    genres.extend(str(val).split('|'))
    
genres = set(genres)

In [ ]:
# number of different genres suggested
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

In [ ]:
print(len(genres))
print(genres)

In [ ]:
len(reviews.user_id.unique())

In [ ]:
reviews.rating.isnull().sum()

In [ ]:
# and even more - open additional cells as necessary
reviews.rating.describe()

In [ ]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 34996,
'The number of ratings in the dataset': 846973,
'The number of different genres': 29,
'The number of unique users in the dataset': 65425,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.3,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10,
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [2]:
movies_url = 'https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat'

In [3]:
reviews_url = 'https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat'

In [4]:
def load_data(movies_url, reviews_url):
    
    movies = pd.read_csv(movies_url, delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, encoding="utf8", engine='python')
    reviews = pd.read_csv(reviews_url, delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, encoding="utf8", engine='python')
    
    genres = []

    for val in movies.genre:
        genres.extend(str(val).split('|'))
    
    genres = set(genres)
    
    return movies, reviews, genres

In [5]:
movies, reviews, genres = load_data(movies_url, reviews_url)

In [6]:
def extract_date(movie):
    date = str(movie)[-5:-1]
    return date

In [7]:
def century_encode(year, century):
    if year in range((century-1)*100, century*100):
        return 1
    else:
        return 0

In [15]:
def clean_movies(movies_df):
    
    movies_df['date'] = movies_df['movie'].apply(extract_date).astype(int)
    movies_df['1800s'] = movies_df['date'].apply(century_encode, args = [19])
    movies_df['1900s'] = movies_df['date'].apply(century_encode, args = [20])
    movies_df['2000s'] = movies_df['date'].apply(century_encode, args = [21])
    
    def split_genres(val):
        try:
            if val.find(gene) >-1:
                return 1
            else:
                return 0
        except AttributeError:
            return 0
    
    for gene in genres:        
        movies_df[gene] = movies_df['genre'].apply(split_genres)
        
    return movies_df

In [17]:
clean_movies = clean_movies(movies)

TypeError: 'DataFrame' object is not callable

In [21]:
clean_movies.head()

,movie_id,movie,genre,date,1800s,1900s,2000s,Adult,Biography,Crime,...,Western,Reality-TV,Romance,Documentary,Music,News,Mystery,Thriller,Short,War
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
def clean_reviews(reviews_df):
    reviews_df['date'] = reviews_df['timestamp'].apply(lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S'))
    
    return reviews_df

In [19]:
clean_reviews = clean_reviews(reviews)

In [20]:
clean_reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-06 05:00:50
1,2,0358273,9,1579057827,2020-01-15 11:10:27
2,2,10039344,5,1578603053,2020-01-10 04:50:53
3,2,6751668,9,1578955697,2020-01-14 06:48:17
4,2,7131622,8,1579559244,2020-01-21 06:27:24


In [ ]:
def save_data(df, database_filename, database_tablename):
    """
    This function loads the cleaned dataframe to SQLite database.
    Arguments:
        df <- Cleaned DataFrame
        database_filename <- filename of the .db file
    """
    engine = create_engine('sqlite:///' + database_filename)
    df.to_sql(database_tablename, engine, index=False)

In [ ]:
def main():
    """
    Main Data Processing function
    This function implement the ETL pipeline:
        1) Data extraction from web
        2) Data cleaning and pre-processing
        3) Data loading to SQLite database
    """
    print(sys.argv)
    if len(sys.argv) == 4:

        movies_url, reviews_url, database_filepath = sys.argv[1:]

        print('Loading data...\n    MOVIES: {}\n    REVIEWS: {}'
              .format(movies_url, reviews_url))
        movies, reviews, genres = load_data(movies_url, reviews_url)

        print('Cleaning data...')
        clean_movies = clean_movies(movies)
        clean_reviews = clean_reviews(reviews)

        print('Saving data...\n    DATABASE: {}'.format(database_filepath))
        save_data(df, database_filepath)

        print('Cleaned data saved to database!')

    else:
        print('Please provide the filepaths of the messages and categories ' \
              'datasets as the first and second argument respectively, as ' \
              'well as the filepath of the database to save the cleaned data ' \
              'to as the third argument. \n\nExample: etl.py ' \
              'https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat' \
              'TwitterMovies.db')


if __name__ == '__main__':
    main()

In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()